# Merging Accounting and Market DD/PD Datasets

This notebook merges the accounting-based and market-based distance-to-default (DD) and probability-of-default (PD) datasets into a single combined dataset.

**Workflow:**
1. Load the latest accounting and market datasets from `data/outputs/datasheet/`
2. Merge on `['instrument', 'year']`
3. Apply clear labeling to distinguish accounting vs market variables
4. Save timestamped merged dataset with archiving (max 5 archives)

In [1]:
# Setup and imports
from pathlib import Path
from datetime import datetime
import pytz
import shutil
import glob
import os
import pandas as pd
import numpy as np

# Find repository root
def find_repo_root(start: Path, marker: str = '.git') -> Path:
    current = start.resolve()
    for candidate in [current, *current.parents]:
        if (candidate / marker).exists():
            return candidate
    return current

base_dir = find_repo_root(Path.cwd())
output_dir = base_dir / 'data' / 'outputs' / 'datasheet'
archive_dir = base_dir / 'archive' / 'datasets'

print(f"Repository root: {base_dir}")
print(f"Output directory: {output_dir}")
print(f"Archive directory: {archive_dir}")

Repository root: /Users/guillaumebld/Documents/Graduate_Research/Professor Abol Jalilvand/fall2025/risk_bank/risk_bank
Output directory: /Users/guillaumebld/Documents/Graduate_Research/Professor Abol Jalilvand/fall2025/risk_bank/risk_bank/data/outputs/datasheet
Archive directory: /Users/guillaumebld/Documents/Graduate_Research/Professor Abol Jalilvand/fall2025/risk_bank/risk_bank/archive/datasets


In [2]:
# Helper functions
def get_timestamp_cdt():
    """Generate timestamp in YYYYMMDD_HHMMSS format (CDT timezone)"""
    cdt = pytz.timezone('America/Chicago')
    return datetime.now(cdt).strftime('%Y%m%d_%H%M%S')

def archive_old_files(output_dir, archive_dir, dataset_type, max_keep=5):
    """Move old files of dataset_type to archive, keeping only max_keep most recent"""
    pattern = str(output_dir / f"{dataset_type}_*.csv")
    old_files = sorted(glob.glob(pattern), key=lambda x: os.path.getmtime(x), reverse=True)
    
    # Move all existing files to archive
    for old_file in old_files:
        archive_path = archive_dir / os.path.basename(old_file)
        shutil.move(old_file, str(archive_path))
        print(f"[ARCHIVE] Moved to archive: {os.path.basename(old_file)}")
    
    # Clean up archive to keep only max_keep files
    archive_pattern = str(archive_dir / f"{dataset_type}_*.csv")
    archive_files = sorted(glob.glob(archive_pattern), key=lambda x: os.path.getmtime(x), reverse=True)
    
    for old_archive in archive_files[max_keep:]:
        os.remove(old_archive)
        print(f"[CLEANUP] Removed old archive: {os.path.basename(old_archive)}")

def get_latest_file(output_dir, dataset_type):
    """Get the most recent file of given dataset_type"""
    pattern = str(output_dir / f"{dataset_type}_*.csv")
    files = sorted(glob.glob(pattern), key=lambda x: os.path.getmtime(x), reverse=True)
    if not files:
        raise FileNotFoundError(f"No {dataset_type} files found in {output_dir}")
    return files[0]

In [3]:
# Load latest accounting and market datasets
accounting_file = get_latest_file(output_dir, 'accounting')
market_file = get_latest_file(output_dir, 'market')

print(f"Loading accounting data from: {os.path.basename(accounting_file)}")
print(f"Loading market data from: {os.path.basename(market_file)}")

df_accounting = pd.read_csv(accounting_file)
df_market = pd.read_csv(market_file)

print(f"\nAccounting dataset: {len(df_accounting)} rows")
print(f"Market dataset: {len(df_market)} rows")

Loading accounting data from: accounting_20251011_042604.csv
Loading market data from: market_20251011_042629.csv

Accounting dataset: 1431 rows
Market dataset: 1305 rows


In [4]:
# Merge datasets on instrument and year
merge_keys = ['instrument', 'year']

# Add prefixes to distinguish variables (except merge keys and final DD/PD)
accounting_cols_to_prefix = [c for c in df_accounting.columns 
                             if c not in merge_keys + ['DD_a', 'PD_a']]
market_cols_to_prefix = [c for c in df_market.columns 
                         if c not in merge_keys + ['DD_m', 'PD_m']]

df_accounting_prefixed = df_accounting.rename(
    columns={c: f'a_{c}' for c in accounting_cols_to_prefix}
)
df_market_prefixed = df_market.rename(
    columns={c: f'm_{c}' for c in market_cols_to_prefix}
)

# Perform outer merge to keep all observations
df_merged = pd.merge(
    df_accounting_prefixed,
    df_market_prefixed,
    on=merge_keys,
    how='outer',
    suffixes=('_a', '_m')
)

# Clean up: Remove any unnamed columns
unnamed_cols = [col for col in df_merged.columns if col.startswith('Unnamed')]
if unnamed_cols:
    print(f'[WARN] Dropping {len(unnamed_cols)} unnamed columns: {unnamed_cols[:5]}...')
    df_merged = df_merged.drop(columns=unnamed_cols)

# Remove duplicate columns (keep first occurrence)
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

print(f"Merged dataset: {len(df_merged)} rows")
print(f"\nColumn count:")
print(f"  Accounting: {len(df_accounting.columns)}")
print(f"  Market: {len(df_market.columns)}")
print(f"  Merged: {len(df_merged.columns)}")

# Show sample
print(f"\nSample merged data:")
display(df_merged[['instrument', 'year', 'DD_a', 'PD_a', 'DD_m', 'PD_m']].head(10))


Merged dataset: 1551 rows

Column count:
  Accounting: 16
  Market: 60
  Merged: 74

Sample merged data:


,instrument,year,DD_a,PD_a,DD_m,PD_m
0,ABCB,2016,16.169315,4.150843e-59,10.347490,2.147999e-25
1,ABCB,2017,17.455646,1.559156e-68,10.495263,4.541225e-26
2,ABCB,2018,16.682782,8.744138e-63,10.192319,1.072919e-24
3,ABCB,2019,7.407534,6.433465e-14,5.073253,1.955359e-07
4,ABCB,2020,11.296038,6.863643e-30,6.419505,6.835930e-11
5,ABCB,2021,7.544877,2.263566e-14,4.520901,3.078854e-06
6,ABCB,2022,7.175878,3.592236e-13,3.844074,6.050423e-05
7,ABCB,2023,9.613568,3.503867e-22,5.679425,6.757405e-09
8,ACNB,2019,17.035894,2.224464e-65,10.447757,7.501612e-26
9,ACNB,2020,15.467251,2.885981e-54,8.615988,3.467036e-18


In [5]:
# Archive old merged files and save new one with timestamp
archive_old_files(output_dir, archive_dir, 'merged', max_keep=5)

timestamp = get_timestamp_cdt()
merged_output = output_dir / f'merged_{timestamp}.csv'
df_merged.to_csv(merged_output, index=False)

print(f"[INFO] Merged dataset saved to: {merged_output}")
print(f"[INFO] Total rows: {len(df_merged)}")
print(f"[INFO] Total columns: {len(df_merged.columns)}")

[ARCHIVE] Moved to archive: merged_20251011_042434.csv
[CLEANUP] Removed old archive: merged_with_exclusions_20251008_013301.csv
[INFO] Merged dataset saved to: /Users/guillaumebld/Documents/Graduate_Research/Professor Abol Jalilvand/fall2025/risk_bank/risk_bank/data/outputs/datasheet/merged_20251011_043202.csv
[INFO] Total rows: 1551
[INFO] Total columns: 74


In [6]:
# Create ESG dataset with DD/PD columns appended
print('[INFO] Creating ESG dataset with DD/PD columns...')

# Load ESG data
esg_file = base_dir / 'data' / 'outputs' / 'datasheet' / 'esg_0718.csv'
if not esg_file.exists():
    esg_file = base_dir / 'data' / 'esg_0718.csv'

if not esg_file.exists():
    print(f'[ERROR] ESG file not found: {esg_file}')
else:
    df_esg = pd.read_csv(esg_file)
    
    # CRITICAL: Remove old DD/PD columns from ESG file if they exist
    old_cols = ['DD_a', 'PD_a', 'DD_m', 'PD_m', 'status']
    old_cols_found = [col for col in old_cols if col in df_esg.columns]
    if old_cols_found:
        print(f'  [WARN] Removing old DD/PD columns from ESG: {old_cols_found}')
        df_esg = df_esg.drop(columns=old_cols_found)
    
    # Clean up any unnamed columns
    unnamed_cols = [col for col in df_esg.columns if col.startswith('Unnamed')]
    if unnamed_cols:
        print(f'  [WARN] Dropping {len(unnamed_cols)} unnamed columns from ESG')
        df_esg = df_esg.drop(columns=unnamed_cols)
    
    print(f'  Loaded ESG data: {len(df_esg)} rows, {len(df_esg.columns)} columns')
    
    # Extract DD/PD columns from merged dataset
    dd_pd_data = df_merged[['instrument', 'year', 'DD_a', 'PD_a', 'DD_m', 'PD_m']].copy()
    
    # Remove duplicates from DD/PD data (keep first occurrence)
    before_dedup = len(dd_pd_data)
    dd_pd_data = dd_pd_data.drop_duplicates(subset=['instrument', 'year'], keep='first')
    after_dedup = len(dd_pd_data)
    if before_dedup > after_dedup:
        print(f'  [INFO] Removed {before_dedup - after_dedup} duplicate DD/PD rows')
    
    # Merge ESG data with DD/PD (now clean, no conflicts)
    df_esg_dd = pd.merge(
        df_esg,
        dd_pd_data,
        on=['instrument', 'year'],
        how='left'
    )
    
    # Final cleanup: remove any unnamed columns
    unnamed_cols = [col for col in df_esg_dd.columns if col.startswith('Unnamed')]
    if unnamed_cols:
        print(f'  [WARN] Dropping {len(unnamed_cols)} unnamed columns after merge')
        df_esg_dd = df_esg_dd.drop(columns=unnamed_cols)
    
    # Remove duplicate rows (keep first)
    before_dedup = len(df_esg_dd)
    df_esg_dd = df_esg_dd.drop_duplicates(subset=['instrument', 'year'], keep='first')
    after_dedup = len(df_esg_dd)
    if before_dedup > after_dedup:
        print(f'  [INFO] Removed {before_dedup - after_dedup} duplicate rows from final dataset')
    
    print(f'  Merged ESG+DD/PD: {len(df_esg_dd)} rows, {len(df_esg_dd.columns)} columns')
    print(f'  New columns: DD_a, PD_a, DD_m, PD_m')
    
    # Archive old ESG+DD files
    archive_old_files(output_dir, archive_dir, 'esg_dd_pd', max_keep=5)
    
    # Save with timestamp
    esg_output = output_dir / f'esg_dd_pd_{timestamp}.csv'
    df_esg_dd.to_csv(esg_output, index=False)
    
    print(f'\n[INFO] ESG+DD/PD dataset saved to: {esg_output}')
    print(f'[INFO] Sample data:')
    
    # Display only columns that exist
    display_cols = ['instrument', 'year']
    for col in ['lnta', 'esg_score', 'DD_a', 'PD_a', 'DD_m', 'PD_m']:
        if col in df_esg_dd.columns:
            display_cols.append(col)
    display(df_esg_dd[display_cols].head(10))


[INFO] Creating ESG dataset with DD/PD columns...
  [WARN] Removing old DD/PD columns from ESG: ['DD_a', 'PD_a', 'DD_m', 'PD_m', 'status']
  Loaded ESG data: 1424 rows, 32 columns
  [INFO] Removed 127 duplicate DD/PD rows
  Merged ESG+DD/PD: 1424 rows, 36 columns
  New columns: DD_a, PD_a, DD_m, PD_m
[ARCHIVE] Moved to archive: esg_dd_pd_20251011_042434.csv
[CLEANUP] Removed old archive: esg_dd_pd_20251004_051328.csv

[INFO] ESG+DD/PD dataset saved to: /Users/guillaumebld/Documents/Graduate_Research/Professor Abol Jalilvand/fall2025/risk_bank/risk_bank/data/outputs/datasheet/esg_dd_pd_20251011_043202.csv
[INFO] Sample data:


,instrument,year,lnta,esg_score,DD_a,PD_a,DD_m,PD_m
0,JPM,2016,14.728184,81.521252,11.422373,1.616415e-30,6.268524,1.822427e-10
1,JPM,2017,14.745152,82.353064,12.014904,1.483567e-33,6.208168,2.680295e-10
2,JPM,2018,14.779651,80.046519,10.890277,6.412505e-28,5.877820,2.078521e-09
3,JPM,2019,14.804077,83.907682,10.173661,1.299758e-24,6.445351,5.766643e-11
4,JPM,2020,15.034793,85.545384,12.565140,1.641474e-36,6.314757,1.352930e-10
5,JPM,2021,15.135549,82.867509,8.535697,6.965624e-18,4.699618,1.303246e-06
6,JPM,2022,15.114542,78.672800,9.835565,3.956130e-23,4.670777,1.500313e-06
7,JPM,2023,15.170158,79.512383,7.353921,9.623768e-14,4.130434,1.810397e-05
8,BAC,2016,14.598529,69.918468,9.078653,5.496324e-20,5.386901,3.584151e-08
9,BAC,2017,14.640227,75.384317,8.031137,4.828675e-16,4.205528,1.302365e-05


In [7]:
# Summary of ESG+DD/PD dataset
if 'df_esg_dd' in locals():
    print('=== ESG+DD/PD DATASET SUMMARY ===')
    print(f'\nTotal observations: {len(df_esg_dd)}')
    print(f'Observations with DD_a: {df_esg_dd["DD_a"].notna().sum()}')
    print(f'Observations with DD_m: {df_esg_dd["DD_m"].notna().sum()}')
    print(f'Observations with both DD_a and DD_m: {df_esg_dd[["DD_a", "DD_m"]].dropna().shape[0]}')
    print(f'\nUnique instruments: {df_esg_dd["instrument"].nunique()}')
    print(f'Year range: {df_esg_dd["year"].min()} - {df_esg_dd["year"].max()}')


=== ESG+DD/PD DATASET SUMMARY ===

Total observations: 1424
Observations with DD_a: 1290
Observations with DD_m: 1290
Observations with both DD_a and DD_m: 1290

Unique instruments: 244
Year range: 2016 - 2023


In [8]:
# Summary statistics
print("=== MERGED DATASET SUMMARY ===")
print(f"\nObservations with both DD_a and DD_m: {df_merged[['DD_a', 'DD_m']].dropna().shape[0]}")
print(f"Observations with only DD_a: {df_merged['DD_a'].notna().sum() - df_merged[['DD_a', 'DD_m']].dropna().shape[0]}")
print(f"Observations with only DD_m: {df_merged['DD_m'].notna().sum() - df_merged[['DD_a', 'DD_m']].dropna().shape[0]}")

print(f"\nDD_a statistics:")
print(df_merged['DD_a'].describe())

print(f"\nDD_m statistics:")
print(df_merged['DD_m'].describe())

print(f"\nPD_a statistics:")
print(df_merged['PD_a'].describe())

print(f"\nPD_m statistics:")
print(df_merged['PD_m'].describe())

=== MERGED DATASET SUMMARY ===

Observations with both DD_a and DD_m: 1417
Observations with only DD_a: 0
Observations with only DD_m: 0

DD_a statistics:
count    1417.000000
mean       13.083009
std         5.635417
min        -6.042400
25%         9.793822
50%        11.768417
75%        15.365677
max        59.282332
Name: DD_a, dtype: float64

DD_m statistics:
count    1417.000000
mean        7.759768
std         3.840419
min         0.899636
25%         5.514719
50%         7.139359
75%         8.964577
max        35.000000
Name: DD_m, dtype: float64

PD_a statistics:
count    1.417000e+03
mean     8.102017e-04
std      2.665497e-02
min      0.000000e+00
25%      1.390587e-53
50%      2.839007e-32
75%      5.984201e-23
max      1.000000e+00
Name: PD_a, dtype: float64

PD_m statistics:
count     1.417000e+03
mean      4.708101e-04
std       7.880232e-03
min      1.124911e-268
25%       1.557376e-19
50%       4.688373e-13
75%       1.746687e-08
max       1.841571e-01
Name: PD_m, dt